# AE4350 - Bio-inspired Intelligence and Learning Assignment
This is the Main notebook for training and testing the DRL portfolio management system
<br>Created on Thu May 12 13:22:50 2022
<br>@author: Reinier Vos, 4663160-TUD

In [1]:
try:
    from utility_v15 import Agent, UtilFuncs, Statistics
except:
    pass
try:
    from AE4350_Assignment.utility_v15 import Agent, UtilFuncs, Statistics
except:
    pass
import sys
from tqdm import tqdm
from tqdm.notebook import trange
import os
import numpy as np
import pandas as pd
import json
import plotly.graph_objects as go
#In TensorFlow 2.3+, you can disable eager mode anytime using the following method:
import tensorflow as tf
#tf.autograph.set_verbosity(0)
import plotly.graph_objects as pgo
import time
#tf.config.run_functions_eagerly(True)
#tf.config.run_functions_eagerly(False)

import logging
#logging.getLogger("tensorflow").setLevel(logging.ERROR)

seed = 10
np.random.seed(seed)
tf.random.set_seed(seed)
import matplotlib.pyplot as plt
import re

print(tf.__version__)
tf.config.list_physical_devices('GPU')

2.8.0


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [ ]:
### Model summaries
from utility_v13 import Actor, Critic
window_size = 128

convLayers = [[64,4],[64,8],[128,8]]
convLayers = [[64,3]]
convLayers = []
convLayers = [[64,3],[64,3],[128,3],[128,3]]
modelsHyper_dct = {"actor_ts_convLayers":convLayers, # [filter,kernelsize]
                   "actor_ts_maxPoolsize":2, # = 0 means not used
                   "actor_ts_maxPoolstrides":2, # default 1
                   "actor_ts_dLayers":[256,256,128,64], # agent.LAST layer will be concatted 
                   "actor_util_dLayers":[], #[16,8], # agent.LAST layer will be concatted 
                   "actor_comb_dLayers":[], #[128,64,32],
                   "actor_regularizer":1e-14, # currently only applied to dense layers
                   "critic_ts_convLayers":convLayers, # [filter,kernelsize]
                   "critic_ts_maxPoolsize":0, # = 0 means not used
                   "critic_ts_maxPoolstrides":2, # default 1
                   "critic_ts_dLayers":[256,256,128,64], # agent.LAST layer will be concatted 
                   "critic_util_dLayers":[],#[16,8], # agent.LAST layer will be concatted 
                   "critic_comb_dLayers":[], #[128,64,32],
                   "critic_action_dLayers":[],#[16,8], # action probabilities related layer
                   "critic_final_dLayers":[32,16], # layers after states and action added 
                   "critic_regularizer":1e-14, # currently only applied to dense layers
                   "use_batchNorm":False,
}

actor = Actor(window_size,6,3,modelsHyper_dct)
actor.model.summary()
critic = Critic(window_size,6,3,modelsHyper_dct)
critic.model.summary()

#inout = tf.keras.layers.Input(shape = (100,), name = "inout")
class INNER(tf.keras.layers.Layer):
    def __init__(self):
        super(INNER, self).__init__()
        self.linear_1 = tf.keras.layers.Dense(32)
        self.linear_2 = tf.keras.layers.Dense(32)
        self.linear_3 = tf.keras.layers.Dense(1)

    def call(self, inputs):
        x = self.linear_1(inputs)
        x = self.linear_2(x)
        return self.linear_3(x)


inModel = INNER()


inner = tf.keras.layers.Input(shape = (100,), name = "input1")
net = inModel(inner)
out = tf.keras.layers.Dense(units = 10)(net)

innerModel = tf.keras.models.Model(inputs = inner, outputs = out)
innerModel.summary()
#output = tf.keras.layers.Dense(units = 3, activation = "softmax")(innerModel.output)

#outerModel = tf.keras.models.Model(inputs = inout, outputs = output)

#outerModel.summary()

### README
<br> Ensure utility v12 is used!
<br> This version has is the google colab compatible version, change next cell for desired environment

In [3]:
# github link

using_colab = False
pay = False # whether youre using the payed version (i.e. own server)

if using_colab:
    from google.colab import files
    if not pay:
        from google.colab import drive
        import shutil
        drive.mount('/content/drive', force_remount=True)
    !git clone https://github.com/rwvosTUD/AE4350_Assignment.git

In [4]:
checkpoint_dir = "runs/v15/v15_tc5_p1" #v10_reg9_p1" # "colabv11_w"

#convLayers = [[32,8],[32,8],[64,4],[64,4],[128,4],[128,4]]
#convLayers = [[32,5],[32,5],[64,5],[64,5]]
#convLayers = [[64,3],[64,3],[128,3],[128,3]]
#convLayers = [[64,4], [64, 4], [128, 4], [128, 4]]
#convLayers = [[64,16], [64, 16], [128, 32], [128, 32]]
convLayers = []
ts_dlayers = [256,256,256,128,64] #[256,256,128,64]
comb_layers = [128,128,64,64,32] #[128,64,64,32]
critic_final_dLayers = [128,128,64,64,32] #[128,128,64,32]
modelsHyper_dct = {"actor_ts_convLayers":convLayers, # [filter,kernelsize]
                   "actor_ts_maxPoolsize":0, # = 0 means not used
                   "actor_ts_maxPoolstrides":2, # default 1
                   "actor_ts_dLayers":ts_dlayers, # agent.LAST layer will be concatted 
                   "actor_util_dLayers":[], #[16,8], # agent.LAST layer will be concatted 
                   "actor_comb_dLayers":comb_layers,
                   "actor_regularizer":1e-14, # currently only applied to dense layers
                   "critic_ts_convLayers":convLayers, # [filter,kernelsize]
                   "critic_ts_maxPoolsize":0, # = 0 means not used
                   "critic_ts_maxPoolstrides":2, # default 1
                   "critic_ts_dLayers":ts_dlayers, # agent.LAST layer will be concatted 
                   "critic_util_dLayers":[],#[16,8], # agent.LAST layer will be concatted 
                   "critic_comb_dLayers":comb_layers,
                   "critic_action_dLayers":[],#[16,8], # action probabilities related layer
                   "critic_final_dLayers":critic_final_dLayers, # layers after states and action added 
                   "critic_regularizer":1e-14, # currently only applied to dense layers
                   "use_batchNorm":False,
}

agent_dct = {"stateTS_size":64,#128, # i.e. window_size 
             "stateUT_size":6, 
             "batch_size":128,
             "data_extraWindow":1,
             "n_budget":1,
             "is_terminal_threshold":1000,
             "model_hyper": modelsHyper_dct,
             "tanh_scale":80,
             "gamma":0.99,
             "tau":0.001,
             "episode_window": 300,
             "mask_input":False,
}

reward_dct = {"rewardType":7, # NOTICE WHICH REWARD FUNCTION WE USE
              "penalty":0,
              "hold_scale": 17,#90, #40,
              "trade_scale":14,
              "trade_cost":5,
              "max_holds":100, # minimum hardcoded to 100!
              "prob_power":1, 
}
extra_dct = {"EXTRACASH" : reward_dct["trade_cost"], # 100
            "EXPAND" : 5,
            "LAST" : 5,
            "PROFITDIFF" : 200,
            "EXPAND_TIMER":  10,
            "TRADECOST_ACTUAL" : 3,
             "START_OFFSET" : 300, #700
             "VALI_EC":50, # extracash amount for validation
}

data, data_extra = UtilFuncs.get_data("traindata",agent_dct["data_extraWindow"],
                                      colab = using_colab)
l = len(data) - 1

window_size = agent_dct["stateTS_size"]
data_extraWindow =  agent_dct["data_extraWindow"]
agent = Agent(agent_dct, data[window_size],
              checkpoint_dir, reward_dct, extra_dct) 
stats = Statistics(checkpoint_dir, training = True)
stats_val = Statistics(checkpoint_dir, training = False)
print("=== ATTENTION: running model for {} stocks ===".format(agent_dct["n_budget"]))
print(l)
plt.plot(data[window_size+agent.START_OFFSET:(agent.START_OFFSET+window_size+agent.episode_window)])
plt.grid(True)

Exception: Checkpoint directory already exists and is not empty, please adjust

 ## Loading of models
### General loading

In [ ]:
### loading of model
#load_dir = "runs/run3"
#agent.load_models(load_dir,40)
#load_dir = "runs/version3/run6_r5"
if using_colab:
    load_dir = "AE4350_Assignment/prev_runs"
else:
    load_dir = "runs/colab/content/v15_NomaskA2_p2" 
    
load_episode = 1500
#buffer_episode_size = agent_dct["episode_window"]
#agent.load_models(load_dir,load_episode,buffer_episode_size, buffer = True)
agent.load_models(load_dir,load_episode, buffer = True, using_colab = using_colab)
#agent.memory.memory_counter = load_episode*2000

### pretrained actor loading

# actor model loading
actor_load_dir = "actor"
agent.load_models(actor_load_dir,0, critic = False, actor = True)

In [ ]:
growth_buyhold = UtilFuncs.plot_data(agent, data, data_extra, data_extraWindow, window_size, training = True)
data_val, data_extra_val = UtilFuncs.get_data("validationdata", data_extraWindow, colab = using_colab)
l_val = len(data_val)-1
growth_buyhold_val = UtilFuncs.plot_data(agent, data_val, data_extra_val, data_extraWindow, window_size, training = False)

In [ ]:
#from utility_v12 import UtilFuncs
show_figs = False
debug = False

testing = False
#''' # uncomment this when resetting a trial 
reward_lst = []
profitdiff_lst = []
expansions_lst = []
expand_i = 1

history = {}
history["training_profit"] = []
history["training_pratio"]  = []
history["validation_profit"] = []
history["validation_pratio"] = []
history["validation_extraCash"] = []
#'''
deadlock_on = False
timer = 0 
expansion_size = 1 # start expansion_size at 1, will be set to EXPANSION after first

saveIter = 100
start = 0
episode_count = 10000 #300


use_terminateFunc = True
terminateFunc_on = False
#'''
deadlock_probStart = 1/10 #1/6  # exploratory probability hack for actions 1 & 2
decay = 0.90
decay_op = 2-decay
deadlock_prob = deadlock_probStart*(decay**(start)) # if we start later in an epoch
switch_episode = 50
#'''
#extra_explore = False # whether to override the original exploration sheme
#scaling = 0.1 #0.02
tbzip_folder="/content/{0}_z.zip".format(checkpoint_dir)
zip_folder = "/content/{0}".format(checkpoint_dir)

min_t = window_size 
stats.reset_all(agent.n_budget*data[window_size], growth_buyhold)
stats_val.reset_all(agent.n_budget*data_val[window_size], growth_buyhold_val)
print("\n ATTENTION DEADLOCK IS SET TO {0} \n".format(deadlock_on))
prev_terminate = True
for e in range(start,episode_count):
    agent.is_eval = False # training!
    
    if e % saveIter == 0 and e != 0:
        episode_start = window_size
        episode_end = l
        agent.is_eval = True # training!
    else:
        ''' 
        SHUFFLING START 
        ''' 
        
        
        #deadlock_go = False
        if (e-start) > agent.LAST:
            profitdiff_mean = np.mean(profitdiff_lst[-agent.LAST:])

            if profitdiff_mean <= 0:
                # EXPONENTIALLY GROW
                deadlock_prob = min(deadlock_prob*decay_op,deadlock_probStart) 
                timer = 0 # reset
            elif profitdiff_mean > agent.PROFITDIFF:
                # DECAY
                deadlock_prob = max(deadlock_prob*decay,0.0001) 
                timer += 1
            else: 
                timer = 0  # reset 
                
            print("E{0} - Deadlock probability {1} | Timer = {2} | expansions = {3} | Prev mean = {4}".format(e,
                                                                         round(deadlock_prob,3), 
                                                                         timer,
                                                                         expand_i-1,
                                                                         profitdiff_mean))
            if timer >= agent.EXPAND_TIMER: 
                expand_i += 1 
                timer = 0
                expansion_size = agent.EXPAND
                #deadlock_go = True

        
        utils_start = [l,agent.START_OFFSET, agent.episode_window]
        episode_start  = UtilFuncs.get_episodeStart(agent, expand_i, expansion_size, utils_start)
        episode_end = episode_start+agent.episode_window
        print("E{2} - Current window [{0},{1}]".format(episode_start,episode_end,e))
    
    '''
    DEADLOCK SWITCH OFF
    
    '''
    if e == 30:
        deadlock_on = False 
        print("\n ATTENTION DEADLOCK IS SET TO {0} \n".format(deadlock_on))
    
    
    #if e >= switch_episode:
    pbar = tqdm(total=(episode_end-episode_start), position=0, leave=True, bar_format='{l_bar}{bar}| {n_fmt}/{total_fmt} ') # progress bar
    #print("== Episode " + str(e) + "/" + str(episode_count) +" ==")
    
    agent.reset(data[episode_start])
    agent.balance += agent.EXTRACASH
    stats.reset_episode()
    stats.extraCash += agent.EXTRACASH 
    '''
    extra cash!
    '''
    
    
    sold_price = 0
    bought_price = agent.inventory_value
    utils_state = [episode_end, stats.n_holds,stats.n_trades, agent.trade_cost]
    state = UtilFuncs.get_state(agent, data, episode_start, window_size + 1, utils_state)
    
    done = False
    terminate = False
    
    #REWARD TYPE SWITCH EXPERIMENT
    #agent.switch_rewardType(7,switch_episode,e)
    '''
    if not e == start: # not prev_terminate and not e == start:
        if deadlock_on:
            deadlock_prob = deadlock_prob*decay # remove
            print("E{0} - Deadlock probability {1}".format(e,prob))
    '''
    # =============================== ITERATION ======================================================
    for t in range(episode_start,episode_end):

        utils_act = [deadlock_prob,data[t],agent.trade_cost]
        action, action_prob = agent.act(state, utils_act)
        
        # DEADLOCK EXPERIMENT
        action = UtilFuncs.break_deadlock(agent,action,e,utils_act, on = deadlock_on) 
            
        # Hanle action
        flags = [use_terminateFunc, terminateFunc_on]
        utils_hdlAct = [action_prob]
        action, profit,  impossible, terminate, term_msg = UtilFuncs.handle_action(agent, stats, action, data, 
                                                                                   t, flags, utils_hdlAct, training = True)
        
        # terminate if required
        if terminate or t == episode_end-1:
            # terminating i.e. agent.LAST iteration 
            done = True # terminal state has no 'next' so change done variable before use in next lines
            
        # get reward
        if t < data_extraWindow:
            ptn = data_extra[t]
        else: 
            ptn = data[t-data_extraWindow]
        utils_reward = [data[t],data[t-1], data[t+1], action, action_prob[0], stats.n_trades, stats.n_holds, impossible, l, terminate]
        reward = agent.get_reward(agent, profit, utils_reward, done)
        stats.total_reward += reward
        
        # take step
        utils_state = [episode_end, stats.n_holds,stats.n_trades, agent.trade_cost]
        next_state = UtilFuncs.get_state(agent, data, t + 1, window_size + 1, utils_state)
        if e % saveIter != 0 or e == 0: 
            '''
            SMALL WINDOW EXPERIMENT
            '''
            actor_local_loss = agent.step(action_prob, reward, next_state, done)
        state = next_state
        
        if terminate:
            # ensure consistent length of arrays then terminate trial
            if t >= min_t:
                utils_pad = [l,t]
                stats.pad_on_terminate(utils_pad)    
            print("Episode {0} was terminated at {1}/{2} due to {3}".format(e,t-window_size,episode_end-episode_start, term_msg))
            break
            
        # collect 
        utils_saveIter = [profit, reward, actor_local_loss, action, t-episode_start]
        stats.collect_iteration(agent,utils_saveIter)
        
        pbar.set_description("Episode {0}| Portfolio: {1}| Balance: {2}| Inventory: {3}| RewardAcc: {4}".format(int(e),
                                                                                                                UtilFuncs.to_currency(agent.balance+agent.inventory_value),
                                                                                                                UtilFuncs.to_currency(agent.balance),
                                                                                                                UtilFuncs.to_currency(agent.inventory_value),
                                                                                                                UtilFuncs.to_currency(stats.total_reward)))
        pbar.update()
        
    # ============================ END ITERATION =================================================
    print("E{3} - Amount of +trades {0}, all trades {1} | ratio = {2}".format(stats.n_posiProfits, 
                                                                              stats.n_trades, 
                                                                              round(stats.n_posiProfits/max(1,stats.n_trades),3), 
                                                                              e))
    '''
    SMALL WINDOW EXPERIMENT

    BREAK CURRENT EPISODE, WE WONT SAVE ANYWAY
    '''
    profitBuyhold = data[episode_end-1]-data[episode_start]
    #profitRL = agent.balance+agent.inventory_value-data[episode_start]-agent.trade_cost*stats.n_trades-stats.extraCash
    profitRL = agent.balance+agent.inventory_value-data[episode_start]-agent.TRADECOST_ACTUAL*stats.n_trades-stats.extraCash
    profitdiff = profitRL-profitBuyhold
    print("E{3} - RL profit = {0} | Buyhold = {1} | diff = {2} | tradecost = {4}".format(round(profitRL,2),
                                                                round(profitBuyhold,2),
                                                                round(profitdiff,2),
                                                                e,
                                                                #agent.trade_cost*stats.n_trades,
                                                                agent.TRADECOST_ACTUAL*stats.n_trades,
                                                                ))
    print("E{0} - impossibles {1}/{2} = {3}".format(e, stats.n_impossible,
                                             stats.n_1or2,
                                             round(stats.n_impossible/stats.n_1or2,3)))
    reward_lst.append(stats.total_reward)
    profitdiff_lst.append(profitdiff) 
    expansions_lst.append(expand_i)
    '''
    if e % 25 == 0 and e != 0:
        x = np.arange(start,e+1)
        y = np.array(reward_lst)
        coef = np.polyfit(x,y,1)
        plt.plot(x,y)
        plt.plot(x,x*coef[0]+coef[1])
        plt.plot(x,np.zeros((e-start+1)))
    '''
    if max(e-1,0) % saveIter == 0 and e != 0 and e != 1 and not debug:
        if not prev_terminate:
            # we skip one iteration as we assume the zip file creation is done now
            if using_colab and not pay: 
                #!zip -r /content/run1_colab_r3v2.zip /content/run1_colab
                #files.download(tbzip_folder)
                shutil.copy(tbzip_folder, "/content/drive/MyDrive")
                #b = 1
                #print("Attempting download, system sleep initiated")
                #time.sleep(24) # Sleep system for x seconds, to give time for download
                #print("System sleep terminated, continueing...")
                
    if e % saveIter != 0 or e == 0: 
        continue
        

    
    if True: #not terminate:
        stats.collect_episode(agent,e, [])
        history["training_profit"].append(stats.compete[-1])
        history["training_pratio"].append(stats.n_posiProfits/max(1,stats.n_trades))
        
    if e % saveIter == 0 and e != 0: #and not terminate:
        if not debug:
            # plot and save
            agent.save_models(e)
            utils_fig = [l, window_size]
            stats.plot_figure(data, e, utils_fig, show_figs = show_figs)
            stats.save_statistics(e)
            extraHistory = {}
            extraHistory["reward_lst"] = reward_lst
            extraHistory["profitdiff_lst"] = profitdiff_lst
            extraHistory["expansions_lst"] = expansions_lst 

            with open(f'./{checkpoint_dir}/EXTRAhistory.json', 'w') as fp:
                json.dump(extraHistory, fp)
    
            # ================ VALIDATION LOOP ===============================
            stats_val.reset_episode()
            stats_val.extraCash += agent.VALI_EC
            agent.is_eval = True
            agent.reset(data_val[window_size])
            agent.balance += agent.VALI_EC
            for t in trange(window_size,l_val):
                utils_state = [l_val, stats_val.n_holds,stats_val.n_trades, agent.trade_cost]
                state = UtilFuncs.get_state(agent, data_val, t, window_size + 1, utils_state)
                utils_act = [deadlock_prob,data_val[t]]
                action, action_prob = agent.act(state, utils_act)

                flags = [use_terminateFunc, terminateFunc_on]
                utils_hdlAct = [action_prob]
                action, profit,  impossible, _, _ = UtilFuncs.handle_action(agent, stats_val, action, data_val, 
                                                                                           t, flags, utils_hdlAct, training = True)
                utils_saveIter = [profit, 0., 0., action, t-window_size]
                stats_val.collect_iteration(agent,utils_saveIter)

            stats_val.collect_episode(agent, e, [])
            print("Final validation profit = {0} | extra cash = {1} | +/all trades= {2}/{3}".format(round(stats_val.compete[-1],2),
                                                                            round(stats_val.extraCash,2),
                                                                              stats_val.n_posiProfits,
                                                                               stats_val.n_trades,))
            utils_fig = [l_val, window_size]
            stats_val.plot_figure(data_val, e, utils_fig, show_figs = show_figs)
            history["validation_profit"].append(stats_val.compete[-1])
            history["validation_pratio"].append(stats_val.n_posiProfits/max(1,stats_val.n_trades))
            history["validation_extraCash"].append(stats_val.extraCash)
            # ============= END VALIDATION LOOP =============================
            with open(f'./{checkpoint_dir}/e{e}/history.json', 'w') as fp:
                json.dump(history, fp)
            
            
        if using_colab and not debug:
            !zip -r "$tbzip_folder" "$zip_folder"
            print("Zip file created, saved next run")
        

    '''
    EXPERIMENT
    '''
    if terminate:
        prev_terminate = True
    else:
        prev_terminate = False
            
    #if debug and not terminate:
    #    print("E{3} - Amount of impossibles {0} out of trades {1}, ratio = {2}".format(n_impossible, n_1or2, n_impossible/n_1or2, e))

    

In [ ]:
np.shape(state)

### manually saving replay buffer
np.savez_compressed('/content/{0}/buffer_e{1}.npz'.format(checkpoint_dir, e), 
                    a = agent.memory.memory_state,
                    b = agent.memory.memory_nextState, 
                    c = agent.memory.memory_action,
                    d = agent.memory.memory_reward,
                    e = agent.memory.memory_dones)

In [ ]:
len(reward_lst)


# Check validation scores

In [ ]:
extraHistory = {}
extraHistory["reward_lst"] = reward_lst
extraHistory["agent.PROFITDIFF_lst"] = agent.PROFITDIFF_lst

with open(f'./{checkpoint_dir}/EXTRAhistory.json', 'w') as fp:
    json.dump(extraHistory, fp)
    
    

In [ ]:
#plt.plot(agent.PROFITDIFF_lst)
plt.plot(reward_lst[:500])

In [ ]:
#load_dir =  "runs/colab/content/colabv10_w150"
load_dir =  "runs/colab/content/v10_reg9_p1"
files = os.listdir(load_dir)
#maxEps = max([list(map(int, re.findall('\d+', file))) for file in files])[0]
maxEps = 159 #90
with open(f'./{load_dir}/e{maxEps}/history.json', 'r') as f:
    history = json.load(f)

plt.figure()
plt.plot(history['training'])
plt.figure()
plt.plot(history['validation'])

In [ ]:
files

In [ ]:
history = {}
history["validation"] = []
# ================ VALIDATION LOOP ===============================
load_dir =  "runs/colab/content/v10_test_p2"
files = os.listdir(load_dir)
files = [list(map(int, re.findall('\d+', file))) for file in files]
files.sort()

files = files[2:] # skip first two

'''
RESET THE SLICING TO [2:] NOT [3:]

'''
episodes = [x[0] for x in files]

agent.EXTRACASH = 0
HORIZON = 0

#episodes = episodes[-1:]

for e in episodes:
    agent.load_models(load_dir,e)
    stats_val.reset_episode()
    agent.is_eval = True
    agent.reset(data_val[window_size])
    
    #agent.balance = agent.EXTRACASH
    #stats_val.agent.EXTRACASH = agent.EXTRACASH
    
    for t in trange(window_size,l_val):
        utils_state = [l_val, stats_val.n_holds,stats_val.n_trades, agent.trade_cost]
        state = UtilFuncs.get_state(agent, data_val, t, window_size + 1, utils_state)
        utils_act = [prob,data_val[t]]
        action, action_prob = agent.act(state, utils_act)

        flags = [use_terminateFunc, terminateFunc_on]
        utils_hdlAct = [action_prob]
        action, profit,  impossible, terminate, term_msg = UtilFuncs.handle_action(agent, stats_val, action, data_val, 
                                                                                   t, flags, utils_hdlAct, training = False)
        utils_saveIter = [profit, 0., 0., action, t-window_size]
        stats_val.collect_iteration(agent,utils_saveIter)
        
        #if action == 2 and agent.balance < np.min(data_val[(t+1):(t+HORIZON)]):
        '''
        if not bool(agent.inventory) and agent.balance < np.min(data_val[(t+1):(t+HORIZON)]): 
            #print(t)
            #print("yes")
            n_mins = sum(data_val[(t+1):] < agent.balance) # amount of times there is an opportunity to return to market
            if n_mins <= 5:
                stats_val.agent.EXTRACASH += data_val[t+1] - agent.balance # extra cash required for purchase
                stats_val.xtr_ind.append(t+1)
                agent.reset(data_val[t+1]) # reset the portfolio
        '''
        
        if not bool(agent.inventory) and agent.balance < data_val[t] and np.argmax(action_prob) == 1:
        #if not bool(agent.inventory) and np.argmax(action_prob) == 1 and all(agent.balance < data_val[(t):(t+HORIZON)]):
            #agent.EXTRACASHreq = data_val[t+1] - agent.balance
            stats_val.agent.EXTRACASH += data_val[t] - agent.balance # extra cash required for purchase
            stats_val.xtr_ind.append(t)
            agent.reset(data_val[t]) # reset the portfolio, as if we did buy it
            #agent.balance += agent.EXTRACASHreq

        #if t < 490 and t > 475:
        #    print("a={0}/b={1}/t={2}".format(action, agent.balance, t))

    stats_val.collect_episode(agent, e, [])
    print("Final validation profit = {0} & extra cash = {1}".format(round(stats_val.compete[-1],2), 
                                                                    round(stats_val.agent.EXTRACASH,2)))
    utils_fig = [l_val, window_size]
    stats_val.plot_figure(data_val, e, utils_fig, show_figs = show_figs)
    history["validation"].append(stats_val.compete[-1])
    # ============= END VALIDATION LOOP =============================
    print("done with episode {}".format(e))
with open(f'./{load_dir}/results/validation_redone.json', 'w') as fp:
    json.dump(history, fp)
    
    
# PLOTTING
x = np.arange(0,len(episodes))
y= np.array(history["validation"])
coef = np.polyfit(x,y,1)
plt.plot(np.array(episodes),np.array(history["validation"]))
plt.plot(np.array(episodes),np.arange(len(episodes))*coef[0]+coef[1])
plt.plot(np.array(episodes),np.zeros((len(episodes))))

In [ ]:
#episodes = episodes[-10:]
x = np.arange(0,len(episodes))
y= np.array(history["validation"])
coef = np.polyfit(x,y,1)
plt.plot(np.array(episodes),np.array(history["validation"]))
plt.plot(np.array(episodes),np.arange(len(episodes))*coef[0]+coef[1])
plt.plot(np.array(episodes),np.zeros((len(episodes))))

In [ ]:
import matplotlib.pyplot as plt
plt.plot(np.array(episodes),np.array(history["validation"][82:]))

In [ ]:
stats_val.reset_episode()
agent.is_eval = True
agent.reset(data_val[window_size])
for t in trange(window_size,l_val):
    utils_state = [l_val, stats_val.n_holds,stats_val.n_trades]
    state = UtilFuncs.get_state(agent, data_val, t, window_size + 1, utils_state)
    utils_act = [prob,data_val[t]]
    action, action_prob = agent.act(state, utils_act)

    flags = [use_terminateFunc, terminateFunc_on]
    action, profit,  impossible, terminate, term_msg = UtilFuncs.handle_action(agent, stats_val, action, data_val, 
                                                                               t, flags, training = False)
    utils_saveIter = [profit, 0., 0., action]
    stats_val.collect_iteration(agent,utils_saveIter)

stats_val.collect_episode(agent, e, [])
utils_fig = [l_val, window_size]
stats_val.plot_figure(data_val, e, utils_fig, show_figs = show_figs)

In [ ]:
dd = agent.memory
with open(f'./memoryBIG.npy', 'wb') as f:
    np.save(f, dd)


In [ ]:

stats_val.collect_episode(agent, e, [])
utils_fig = [l_val, window_size]
stats_val.plot_figure(data_val, e, utils_fig, show_figs = show_figs)

In [ ]:
# final save
agent.save_models(0)
if using_colab:
    #!zip -r /content/run1_colab_r3v2.zip /content/run1_colab
    !zip -r "$tbzip_folder" "$zip_folder"
    files.download(tbzip_folder)

In [ ]:
# Manually save results
'''
with open(f'./models/{runName}/{runName}_parameters.txt', 'w') as f:
    print(allParameters, file=f)

# Saving results
with open(f'./{checkpoint_dir}/results/total_rewards.npy', 'wb') as f:
    np.save(f, np.array(total_reward_list))
with open(f'./{checkpoint_dir}/results/EveryProfit_dct.json', 'w') as fp:
    json.dump(EveryProfit_dct, fp)
'''

In [ ]:
# loading results
'''
with open(f'./{checkpoint_dir}/total_rewards.npy', 'rb') as f:
    total_reward_list = np.load(f)
with open(f'./{checkpoint_dir}/EveryProfit_dct.json', 'r') as fp:
    EveryProfit_dct = json.load(fp)
'''

In [ ]:
# Setup naive benchmark, buy and hold for training data
test_data, test_data_extra = UtilFuncs.get_data("testdata", data_extraWindow)
data = test_data
data_extra = test_data_extra
fig = pgo.Figure()
fig.update_layout(showlegend=True)
fig.add_trace(pgo.Scatter(x=np.arange(len(test_data)), y=test_data,
                    mode='lines',
                    name='stock growth'))
fig.add_trace(pgo.Scatter(x=np.arange(len(test_data_extra)), y=test_data_extra,
                    mode='lines',
                    name='stock MA adjusted W ={}'.format(data_extraWindow)))
fig.show()
growth_buyhold_per = (data[-1]-data[window_size])/data[window_size]

print("Naive buy & hold strategy on TEST data has a portfolio growth of {0}% per asset bought at the start of 2015 and sold at 2018".format(round(growth_buyhold_per,3)))
growth_buyhold_cash = agent.n_budget*data[window_size]*growth_buyhold_per
growth_buyhold = (agent.n_budget*(data-data[window_size]))[window_size:-1]
print("For current budget of {0}, this means that for {1} stocks bought in 2015 and sold in 2018 one achieves a final portfolio growth of {2}".format(UtilFuncs.to_currency(agent.n_budget*data[window_size]),agent.n_budget,UtilFuncs.to_currency(growth_buyhold_cash)))

In [ ]:
#load_dir = "runs/colab/content/run1_colab_noPTcont"
load_dir = "runs/colab/content/run1_v6_prftAdj+holdbonus25"
try:
    agent.setup_validation(load_dir)
except:
    pass
checkpoint_dir = load_dir
val_episode = 20
agent.load_models(load_dir,val_episode)
agent.load_attributes(load_dir)

In [ ]:
# Validation cell
data = test_data # sanity check
data_extra = test_data_extra
l = len(data) - 1
l1 = 2263
agent.is_eval = True #True
agent.EXTRACASH = 0
showfigs = True
deadlock_on = False # must remain off!
debug = False
use_terminateFunc = True
terminateFunc_on = False
prob = 0 # not used
# initialize and container setup
agent.reset(data[window_size])
agent.validation_agent.EXTRACASH(agent.EXTRACASH) # function must be after reset!
total_reward = 0 # total profit resets every epsiode 
n_trades = 0 
n_impossible = 0 
n_holds = 0 
n_1or2 = 1 # 1 not zero because we cant have division by zero 
profits = []
balances = []
rewards = []
inventories = [] # inventory value (only stocks)
actor_local_losses = []
buy_ind = []
sell_ind = []
utils_state = [l1, n_holds, n_trades]
state = UtilFuncs.get_state(agent, data, window_size, window_size + 1, utils_state)
pt1 = data_extra[-1]

pbar = tqdm(total=l, position=0, leave=True, bar_format='{l_bar}{bar}| {n_fmt}/{total_fmt} ') # progress bar
fig = pgo.Figure()
fig.update_layout(showlegend=True)
done = False
terminate = False

for t in range(window_size,l):
    profit = 0 # initialize
    change = 0 
    impossible = False
    utils_act = [prob,data[t]]
    action, action_prob = agent.act(state, utils_act)

    # DEADLOCK EXPERIMENT
    #action = UtilFuncs.break_deadlock(agent,action,e,utils_act, on = deadlock_on) 

    # =========================== ACTION SPACE ======================================
    if action == 0:
        n_holds += 1

    elif action == 1:
        n_1or2 += 1
        
        if len(agent.inventory) == 0: #max one stock 
            if agent.balance > data[t]:
                # BUYING stock, only if there is balance though
                agent.inventory.append(data[t])
                sold_price = agent.inventory_conj.pop(0)

                profit = sold_price - data[t]

                change = -data[t]
                buy_ind.append(t)
                n_trades += 1
                n_holds = 0 # reset counter
            else: 
                agent.EXTRACASH += data[t] - agent.balance
                agent.reset(data[t])
        else:
            impossible = True
            n_impossible += 1
            n_holds += 1 # effectively no buy is a hold
            if not use_terminateFunc:
                terminate = True

    elif action == 2:
        n_1or2 += 1
        if len(agent.inventory) > 0: 
            # SELLING stock, only if there are stocks held

            bought_price = agent.inventory.pop(0)
            agent.inventory_conj.append(data[t])

            profit = data[t] - bought_price 

            change = data[t]
            sell_ind.append(t)
            n_trades += 1
            n_holds = 0 # reset counter
        else:
            impossible = True
            n_impossible += 1
            n_holds += 1 # effectively no sell is a hold
            if not use_terminateFunc:
                terminate = True
    # =============================== END ======================================

    agent.update_balance(change)
    agent.update_inventory(data[t])
    if use_terminateFunc:
        utils_term = [n_impossible, np.min(data[(t+1):])]
        terminate = agent.check_threshold(utils_term, terminateFunc_on= terminateFunc_on)
        #terminate = agent.check_threshold()
    if terminate or t == l-1:
        # terminating i.e. agent.LAST iteration 
        done = True # terminal state has no 'next' so change done variable before use in next lines


    if t < data_extraWindow:
        ptn = data_extra[t]
    else: 
        ptn = data[t-data_extraWindow]
    utils_reward = [data[t],pt1, ptn, action, n_trades, n_holds, impossible, l, terminate]
    pt1 = data[t]
    '''
    change names of variables if it works
    '''
    reward = agent.get_reward(agent, profit, utils_reward, done)
    total_reward += reward

    utils_state = [l, n_holds,n_trades]
    next_state = UtilFuncs.get_state(agent, data, t + 1, window_size + 1, utils_state)

    #actor_local_loss = agent.step(action_prob, reward, next_state, done)
    state = next_state

    # collect 
    balances.append(agent.balance)
    inventories.append(agent.inventory_value)
    profits.append(profit)
    rewards.append(reward)
    #actor_local_losses.append(float(actor_local_loss))
    
    if terminate:
        # ensure consistent length of arrays then terminate trial
        if t >= 1500:
            balances = np.pad(balances,(0,l-t-1),'constant',constant_values=(0,balances[-1])).tolist()
            inventories = np.pad(inventories,(0,l-t-1),'constant',constant_values=(0,inventories[-1])).tolist()
            profits = np.pad(profits,(0,l-t-1),'constant',constant_values=(0,0)).tolist()
            rewards = np.pad(rewards,(0,l-t-1),'constant',constant_values=(0,0)).tolist()
            #actor_local_losses = np.pad(actor_local_losses,(0,l-t-1),'constant',constant_values=(0,actor_local_losses[-1])).tolist()
        print("Episode {0} was terminated at {1}/{2} due to terminal state".format('test',t,l))
        break

    pbar.set_description("VALIDATION e{4}| Portfolio: {0}| Balance: {1}| Inventory: {2}| RewardAcc: {3}".format(UtilFuncs.to_currency(agent.balance+agent.inventory_value),
                                                                                                            UtilFuncs.to_currency(agent.balance),
                                                                                                            UtilFuncs.to_currency(agent.inventory_value),
                                                                                                            UtilFuncs.to_currency(total_reward),
                                                                                                            val_episode))
    pbar.update()

        
    

    # collect & save
growth = (np.array(balances)+np.array(inventories)-agent.budget).tolist() # 
compete = (np.array(growth)-growth_buyhold).tolist() # compete vs buyhold
print("Amount of impossibles {0}, trades {1} (+1!)| ratio = {2}".format(n_impossible, n_1or2, n_impossible/n_1or2))
fig.update_layout(showlegend=True, xaxis_range=[window_size, l], 
                  title_text = "Test e{5} | profit RL: {0} vs buyhold: {1}, difference = {2} | extra cash = {3}| Impossible ratio {4}".format(round(growth[-1],2),
                                                                                                                     round(growth_buyhold[-1],2),
                                                                                                                     round(growth[-1]-growth_buyhold[-1],2),
                                                                                                                     agent.EXTRACASH,
                                                                                                                     round(n_impossible/n_1or2,3),
                                                                                                                     val_episode))
fig.add_trace(pgo.Scatter(x=np.arange(len(data)), y=data,
                    mode='lines',
                    name='data'))
fig.add_trace(pgo.Scatter(x=buy_ind, y=data[buy_ind], marker_color = "green",
                    mode='markers',
                    name='buy'))
fig.add_trace(pgo.Scatter(x=sell_ind, y=data[sell_ind], marker_color = "red",
                    mode='markers',
                    name='sell'))
if showfigs:
    fig.show(render = "browser")
fig.write_html("./{0}/validation/e{1}_tradesTest.html".format(checkpoint_dir,val_episode))
    
# collect & save
save_df = {"balance":balances,"inventory":inventories,"profit":profits,"reward":rewards,"growth":growth,"compete":compete,"growth_buyhold":growth_buyhold.tolist()}
save_df = pd.DataFrame(save_df).to_csv("./{0}/validation/e{1}_testResults.csv".format(checkpoint_dir,val_episode))
print("Succesfully saved csv file")



In [ ]:
agent.EXTRACASH

In [ ]:
utils_state = [l, n_holds]
t = 99
state = UtilFuncs.get_state(agent, data, t, window_size + 1, utils_state)
utils_act = [prob,data[t]]
state[0][0:] = 0.5 
state[0][-5] = -0.2 # bal
state[0][-4] = -0.3 # invent
state[0][-3] = -0.3 # port
state[0][-2] = 0.8 # hold
state[0][-1] = 0. # holdin
print(state[0])

In [ ]:
action, action_prob = agent.act(state, utils_act)
action_prob

In [ ]:
## Plotting RL vs buyhold
fig = pgo.Figure()
fig.update_layout(showlegend=True)
fig.add_trace(pgo.Scatter(x=np.arange(len(growth)), y=np.array(growth),
                    mode='lines',
                    name='growth_RL'))
fig.add_trace(pgo.Scatter(x=np.arange(len(growth)), y=np.array(growth_buyhold),
                    mode='lines',
                    name='growth_buyhold'))
fig.add_trace(pgo.Scatter(x=buy_ind, y=data[buy_ind], marker_color = "green",
                    mode='markers',
                    name='buy'))
fig.add_trace(pgo.Scatter(x=sell_ind, y=data[sell_ind], marker_color = "red",
                    mode='markers',
                    name='sell'))
fig.show()

# testing whether model even learnt
model1 = Agent(window_size, batch_size)
model1 = model1.critic_target.model
a= model1.get_weights()
print(a[10])
print("=================================")
model2 = Agent(window_size, batch_size)
model2.critic_target.model.load_weights("./run1/e203/critic_local.h5")
b =model2.critic_target.model.get_weights()
print(b[10])

In [ ]:
growth_buyhold[100]